<a href="https://colab.research.google.com/github/Swagat-modder/DL-NLP-AI-projects/blob/main/Next_word_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re

In [2]:
def file_to_sentence_list(file_path):
    with open(file_path, 'r') as file:
        text = file.read()

    # Splitting the text into sentences using
    # delimiters like '.', '?', and '!'
    sentences = [sentence.strip() for sentence in re.split(
        r'(?<=[.!?])\s+', text) if sentence.strip()]

    return sentences

file_path = '/content/pizza.txt'
text_data = file_to_sentence_list(file_path)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
    input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Convert target data to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)


In [3]:
# Defining the model
model = Sequential()
model.add(Embedding(total_words, 10,
                    input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [4]:
# Train the model
model.fit(X, y, epochs=500, verbose=1)


Epoch 1/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - accuracy: 0.0426 - loss: 6.3719
Epoch 2/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - accuracy: 0.0554 - loss: 5.7658
Epoch 3/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.0458 - loss: 5.6884
Epoch 4/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step - accuracy: 0.0569 - loss: 5.7044
Epoch 5/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - accuracy: 0.0555 - loss: 5.6831
Epoch 6/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.0574 - loss: 5.6428
Epoch 7/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.0689 - loss: 5.5108
Epoch 8/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 66ms/step - accuracy: 0.0623 - loss: 5.5219
Epoch 9/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - accuracy: 0.0687 - loss: 5.3814
Epoch 10/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.0709 - loss: 5.3130
Epoch 11/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.0732 - loss: 5.2492
Epoch 12/500
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step

In [7]:
# Generate next word predictions
seed_text = "Furthurmore"
next_words = 5

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted_probs = model.predict(token_list)
    predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
    seed_text += " " + predicted_word

print("Next predicted words:", seed_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Next predicted words: Furthurmore pizza has become a symbol
